In [2]:
import pandas as pd
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
cider_scorer = Cider()
bleu_scorer = Bleu(4)
meteor_scorer = Meteor()
rouge_scorer = Rouge()

In [27]:
# Load the CSV files
vgg19_path = 'model_vgg19_10_prediction_IMERIT_2024_03_25_TEST_v5.csv'
rn152_path = 'model_resnet152_10_prediction_IMERIT_2024_03_25_TEST_v5.csv'
refer_path = 'metadata_IMERIT_2024_03_25_TEST_v5.csv'

# Read the files
vgg19_predictions = pd.read_csv(vgg19_path).rename(columns={'text': 'vgg19',})
rn152_predictions = pd.read_csv(rn152_path).rename(columns={'text': 'rn152',})
groundtruths = pd.read_csv(refer_path).rename(columns={'text': 'target',})

# Merging the dataframes on the filenames
merged_df = pd.merge(vgg19_predictions, rn152_predictions, left_on='file_name', right_on='file_name')
merged_df = pd.merge(merged_df, groundtruths , left_on='file_name', right_on='file_name')


In [28]:
merged_df 

,file_name,vgg19,rn152,target
0,7C67AB4E289F131709410245_1709410245332_13_1_0_...,a young boy standing next to a fire hydrant on...,a man standing next to a dog on the ground wit...,A man checks his phone before leaving the yard.
1,7C67AB4F513F131699029751_1699029753718_13_1_0_...,a group of people standing next to each other ...,a group of people sitting on a bench in front ...,A person carries a package indoors.
2,7C67AB4253DA131692816209_1692816209108_13_1_0_...,a person standing in the middle of a field wit...,a black bird sitting on top of a green grass c...,A person is leaving the house.
3,7C67AB0DF159131701206089_1701206089308_13_1_0_...,a person standing next to a bed with a bag,there is a man that is sitting on a bench in t...,A delivery person drops off a package.
4,7C67AB0E44A6131692078368_1692078368805_13_1_0_...,an airplane that is sitting in the middle of a...,there is an image of a lot of luggage that are...,A person is picking up their delivered package.
...,...,...,...,...
977,7C67AB0E1A8A131686442916_1686442916154_13_1_0_...,a man standing in front of a truck with a man ...,a man standing in front of a truck with a truck,A man walks toward a parked vehicle.
978,7C67AB0D196B131709354595_1709354595864_13_1_0_...,a red and white car parked in front of a window,a group of people standing next to a red car,A person ran across the parking lot.
979,7C67AB431F38131697215491_1697215491604_13_1_0_...,a fire hydrant in the middle of a city street,a man standing in front of a boat in a cage,Residents cleaning and organizing items in the...
980,7C67AB0F3288131704286497_1704286497550_13_1_0_...,a woman sitting on the back of a car with a car,a cat sitting on top of a car next to a car an...,A person walks up the driveway carrying a bag.


In [46]:

for model_name in ['vgg19', 'rn152']:
    print('-'*20, model_name.upper(), '-'*20)
    refer_captions = {}
    predc_captions = {}
    for i in range(len(merged_df)):
        refer_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i][model_name])]
        predc_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i]['target'])]
    c, _ = cider_scorer.compute_score(refer_captions, predc_captions); print("CIDEr", c)
    b, _ = bleu_scorer.compute_score(refer_captions, predc_captions); print('BLEU', b)
    r, _ = rouge_scorer.compute_score(refer_captions, predc_captions); print('ROUGE', r, '\n')

-------------------- VGG19 --------------------
CIDEr 0.036490947241140666
{'testlen': 7167, 'reflen': 11664, 'guess': [7167, 6185, 5203, 4221], 'correct': [959, 51, 0, 0]}
ratio: 0.6144547325102354
BLEU [0.07144620078661279, 0.01773592217667285, 3.184079383057296e-08, 4.4952987680319783e-11]
ROUGE 0.09457508711908663 

-------------------- RN152 --------------------
CIDEr 0.038019906963850575
{'testlen': 7167, 'reflen': 11985, 'guess': [7167, 6185, 5203, 4221], 'correct': [984, 44, 3, 1]}
ratio: 0.597997496871039
BLEU [0.07009776689995645, 0.01595628637425042, 0.0042162451443005285, 0.0017352105308910395]
ROUGE 0.09702593981792966 

